# Histórico de cotações de fechamento Ptax de compra e venda do BCB (Banco Central do Brasil)
Este notebook tem como objetivo automatizar o processo de obtenção das cotações de fechamento Ptax de compra e venda de diversas moedas disponibilizadas pelo BCB, incluindo também a paridade com relação ao Dólar dos EUA.
Conforme explicitado na página de origem, os dados são provenientes de Refinitiv.

###### Aviso importante: os dados obtidos servem apenas para fins de consulta e estudo, podendo não necessariamente refletir a realidade ou apresentarem inconsistências. 

### Importação de bibliotecas
As bibliotecas utilizadas são basicamente Numpy, Pandas e Date Time. 
O SQLite é utilizado para testes, mas não influencia na saída de dados diretamente.

In [13]:
import sqlite3
import numpy as np
import pandas as pd
import datetime

### Obtenção de dados
Os dados são obtidos através do arquivo .csv disponibilizado para download na página https://www.bcb.gov.br/estabilidadefinanceira/historicocotacoes. Crio a string com o URL de download, contendo o código da moeda e as datas iniciais e finais - que precisam estar em um período máximo de 6 meses. 

In [14]:
def cotacao_periodo(moeda,inicial,final):
    url = "https://ptax.bcb.gov.br/ptax_internet/consultaBoletim.do?method=gerarCSVFechamentoMoedaNoPeriodo&ChkMoeda="+str(moeda)+"&DATAINI="+inicial+"&DATAFIM="+final
    
    colunas = ["data_cotacao",
               "codigo_moeda",
               "tipo_moeda",
               "simbolo",
               "taxa_compra",
               "taxa_venda",
               "paridade_compra",
               "paridade_venda"]
    
    tipos = {"data_cotacao":str,
             "codigo_moeda":str,
             "tipo_moeda":str,
             "simbolo":str,
             "taxa_compra":float,
             "taxa_venda":float,
             "paridade_compra":float,
             "paridade_venda":float}
    
    csv = pd.read_csv(url, header = 0, encoding="ISO-8859-1",
                      sep = ";", decimal = ",", names = colunas, dtype = tipos)
    
    csv.loc[:,"data_cotacao"] = pd.to_datetime(csv["data_cotacao"].str.zfill(8), format="%d%m%Y")
    
    return csv

Antes de carregar a tabela, obtemos a data máxima disponível na tabela de registro de cargas. Assim, essa data será utilizada como base para o parâmetro de data inicial da URL que obtem os dados de cotações. A data final, por sua vez, será o dia atual.

In [15]:
conexao = sqlite3.connect("cotacao_dw.db")
cursor = conexao.cursor()

In [16]:
data_inicial = pd.read_sql("""
    SELECT STRFTIME('%d/%m/%Y',MAX(data_cotacao)) data_inicial 
    FROM fat_taxa""", conexao)["data_inicial"][0]

In [17]:
data_final = datetime.date.today().strftime("%d/%m/%Y")

In [18]:
if data_inicial == None:
    data_inicial = datetime.date.today() + datetime.timedelta(days=-120)
    data_inicial = data_inicial.strftime("%d/%m/%Y")
else:
    pass

Códigos das moedas: 
- Dólar EUA = 61
- Dólar Canadá = 48
- Euro = 222
- Renminbi Chinês = 178
- Renminbi Hong Kong = 2332
- Libra esterlina = 115
- Peso/México = 165
- Peso/Argentina = 156

Para obter os códigos, basta acessar o site do BCB, selecionar a moeda desejada em um período qualquer e verificar o código indicado no endereço de download do .csv da tabela apresentada (parâmetro "ChkMoeda=").

In [19]:
cotacao = pd.DataFrame()

In [20]:
moeda = np.array([61, 222, 178, 2332, 48, 115, 165, 156])

In [21]:
for item in moeda:
    atualizacao = cotacao_periodo(item,data_inicial,data_final)
    cotacao = pd.concat([cotacao,atualizacao])

In [22]:
cotacao = cotacao.sort_values(["simbolo", "data_cotacao"])

In [23]:
cotacao.head(10)

,data_cotacao,codigo_moeda,tipo_moeda,simbolo,taxa_compra,taxa_venda,paridade_compra,paridade_venda
0,2022-10-13,706,A,ARS,0.03503,0.03504,151.30,151.32
1,2022-10-14,706,A,ARS,0.03483,0.03484,151.62,151.63
2,2022-10-17,706,A,ARS,0.03456,0.03456,152.51,152.52
3,2022-10-18,706,A,ARS,0.03448,0.03448,152.82,152.83
4,2022-10-19,706,A,ARS,0.03447,0.03448,153.16,153.17
5,2022-10-20,706,A,ARS,0.03398,0.03399,153.50,153.51
6,2022-10-21,706,A,ARS,0.03375,0.03375,153.82,153.83
7,2022-10-24,706,A,ARS,0.03408,0.03409,154.74,154.75
8,2022-10-25,706,A,ARS,0.03424,0.03424,155.05,155.06
9,2022-10-26,706,A,ARS,0.03426,0.03427,155.40,155.42


In [24]:
cotacao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 21
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   data_cotacao     176 non-null    datetime64[ns]
 1   codigo_moeda     176 non-null    object        
 2   tipo_moeda       176 non-null    object        
 3   simbolo          176 non-null    object        
 4   taxa_compra      176 non-null    float64       
 5   taxa_venda       176 non-null    float64       
 6   paridade_compra  176 non-null    float64       
 7   paridade_venda   176 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 12.4+ KB


### Inserção e atualização no banco de dados
Como etapa experimental, a ideia foi criar uma estrutura de atualização dos dados utilizando uma base desenvolvida em SQLite no notebook "banco_taxas_bcb". 

O código a seguir carrega uma cópia exata do dataframe "cotacao" na tabela "stg_cotacao" do banco de dados. Desta forma, é possível aplicar o "upsert".
Vídeo explicativo que foi utilizado como apoio: https://www.youtube.com/watch?v=1ZVcgg0GWJs

In [25]:
conexao.execute("DELETE FROM stg_cotacao;",)
cotacao.to_sql("stg_cotacao", conexao, if_exists = "append", index = False)

176

#### Carga de tabela "dim_moeda"
A carga desta tabela consiste em uma seleção distinta de três colunas do dataframe "cotacao". Utilizei a função de "upsert" do SQLite para realizar a inserção/atualização de dados com base no que foi obtido do site do BCB.

In [26]:
dim_moeda = """
    INSERT INTO dim_moeda (codigo_moeda, tipo_moeda, simbolo)
    SELECT DISTINCT codigo_moeda, tipo_moeda, simbolo
    FROM stg_cotacao
    WHERE TRUE
    ON CONFLICT (codigo_moeda)
    DO UPDATE SET
        tipo_moeda = excluded.tipo_moeda,
        simbolo = excluded.simbolo;
"""

conexao.execute(dim_moeda)

#### Carga de tabela "fat_taxa"
A tabela carrega todos os dados das taxas reportados no site do BCB. Da mesma maneira que a dimensão "dim_moeda", utilizei o método de "upsert" do SQLite neste caso. 

In [27]:
fat_taxa = """
    INSERT INTO fat_taxa (data_cotacao, codigo_moeda, taxa_compra, taxa_venda, paridade_compra, paridade_venda)
	SELECT 
		data_cotacao,
		codigo_moeda,
		taxa_compra,
		taxa_venda,
		paridade_compra,
		paridade_venda
	FROM stg_cotacao
	WHERE TRUE
	ON CONFLICT (data_cotacao, codigo_moeda)
	DO UPDATE SET
		taxa_compra = excluded.taxa_compra,
		taxa_venda = excluded.taxa_venda,
		paridade_compra = excluded.paridade_compra,
		paridade_venda = excluded.paridade_venda;
"""

conexao.execute(fat_taxa)

#### Carga de tabela "fat_carga"
Esta tabela será a utilizada para registro da execuções de carga por moeda; assim, é possível registrar as datas em que ocorreram as últimas cargas para utilização nos parâmetros da URL.

In [28]:
fat_carga = """
    INSERT INTO fat_carga (data_registro)
	SELECT
		MAX(data_cotacao) data_registro
	FROM stg_cotacao;
"""

conexao.execute(fat_carga)

In [29]:
conexao.commit()
conexao.close()